# Análisis preliminar del conjunto de datos

In [ ]:
# Importaciones de paquetes
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x : '%.4f' % x)
import numpy as np
# Importaciones de unidades de soporte
import sys
sys.path.append("..")
from src import sp_funciones as sp

In [ ]:
df_raw = pd.read_excel("../data/amazon_churn_datosbrutos.xlsx") # Leer el archivo

In [ ]:
df = df_raw.copy() # Hacer una copia del dataframe con el que trabajar

## Análisis exploratorio preliminar

In [ ]:
sp.eda_preliminar(df) # LLamar a la función para el análisis exploratorio de los datos 

## Estandarización de datos

In [6]:
df.columns = df.columns.str.replace(" ","_").str.lower() # Pasar los nombres de las columnas a minúsculas y sin espacios
for col in df.select_dtypes(include='O').columns: # Pasar los nombres de los valores de las columnas categóricas a minúsculas y sin espacios
  df[col] = df[col].apply(lambda x: x.replace(" ","_").lower() if isinstance(x, str) else x)

## Limpieza de datos

In [ ]:
# Eliminar columnas irrelevantes para el objetivo del análisis
columnas_eliminar = ['churned', 'phone_number','hobby','favorite_tv_show', 'internal_notes','churn_label_binario']
df.drop(columns= columnas_eliminar, inplace=True)

In [ ]:
df['contract_type'] = df['contract_type'].str.strip('_').str.replace("-","_", regex=True) # Eliminar espacios y guiones sobrantes de la columna 'contrat_type'

df['monthly_charge'] = df['monthly_charge'].apply(lambda x: float(x.replace("_usd","")) if isinstance(x,str) else x) # Eliminar carácteres sobrantes de 'monthly_charge'

df['unlimited_data_plan'] = df['unlimited_data_plan'].map({1:'yes', 0:'no'}) # Mapear los valores '1' y '0' por 'yes' y 'no' de 'unlimited_data_plan'

df['gender'] = df['gender'].replace('hombre', 'male') # Cambiar los valores 'hombres' por 'male'

df['preferred_contact_method'] = df['preferred_contact_method'].replace('teléfono', 'phone') # Traducción de valores

df['customer_segment'] = df['customer_segment'].replace({'alto':'high','medio':'medium','bajo':'low'}) # Traducción de valores

df['applied_discount'] = df['applied_discount'].map({True:'yes', False:'no'}) # Reemplazar 'true' y 'false' por 'yes' y 'no'

df['contact_date'] = df['contact_date'].astype(str).str.replace("-","/") # Normalización de la fecha de contacto

df['average_monthly_expenses'] = df['average_monthly_expenses'].apply(lambda x: round(x, 2)) # Redondeo de valores en 'average_monthly_expenses'

df = df.rename(columns={'senior':'senior_65'}) # Cambiar el nombe de la columna 'senior' para indicar que se refieren a 65 o más

## Validación de datos

In [ ]:
df['churn_label_binario'] = df['churn_label'].map({'Yes': 1, 'No':0}) 
df['churn_label_binario'].equals(df['churned']) # Comprobar que 'churn_label' y 'churn' son la misma columna pero con el tipo de dato distinto, en ese caso eliminar 'churned'.

# Comprobar que la columna 'under_30' y 'senior' están bien establecidas de acuerdo a la columna 'edad'
validacion_under_30 = df[(df['under_30'] == 'yes') & (df['age'] >= 30)]
errores_under_30_count = validacion_under_30.shape[0]
validacion_senior = df[(df['senior']=='yes') & (df['age'] < 65)]
errores_validacion_senior = validacion_senior.shape[0]

# Validar que en ningún caso 'total charges' sea menor que 'monthly charge'
invalid_charges = df[df["total_charges"] < df["monthly_charge"]]
invalid_charges.shape[0]

In [ ]:
df.to_excel("../data/datos_churn_limpios.xlsx", index=False) # Guardar cambios

# Gestión de nulos 

In [ ]:
df_limpio = pd.read_excel("../data/datos_churn_limpios.xlsx") # Leer el archivo

### Análisis general columnas categóricas

In [ ]:
sp.calcular_nulos(df_limpio) # Observación general de nulos
sp.analisis_general_cat(df_limpio) # Análisis general de las columnas categóricas
sp.subplot_col_cat(df_limpio) # Graficar columnas categoricas

### Gestión de nulos de las columnas categóricas

In [ ]:
# Crear un dataframe solo con las columnas categóricas
df_cat = df_limpio[df_limpio.select_dtypes(include='O').columns]
# Observar los nulos de las columnas categóricas
sp.calcular_nulos(df_cat)

Identificamos tres columnas categóricas con valores nulos: payment_method, churn_category, churn_reason

In [ ]:
# Comprobar si alguna de estas columnas tiene un valor predominante, por si lo usamos para rellenar los nulos
sp.analisis_general_cat(df_cat)

En el caso de 'churn_category' y 'churn_reason', aproximadamente el **74% de los valores están vacíos**. Dado que este porcentaje es muy alto, no tiene sentido rellenarlos con otra categoría, ya que podríamos introducir sesgos o información incorrecta.  

Por otro lado, 'payment_method' tiene pocos valores nulos, pero no existe un método de pago predominante que podamos usar para rellenarlos sin afectar la integridad de los datos.  

Por esta razón, hemos decidido reemplazar los valores nulos en estas columnas con 'unknown', para mantener la consistencia sin distorsionar la información existente.  

In [ ]:
# Reemplazar valores nulos en columnas categóricas con "unknown"
df_cat = df_cat.fillna('unknown')
df_limpio[df_limpio.select_dtypes(include='O').columns] = df_cat
# Verificar si quedan nulos en columnas categóricas después del reemplazo
sp.calcular_nulos(df_limpio)

# Gestión de nulos de las columnas numéricas

In [ ]:
# Crear un dataframe solo con las columnas numericas
df_num = df.select_dtypes(include= np.number).columns.to_list()

## Gestión de valores atípicos
Antes de gestionar los valores nulos, trabajaremos los valores atípicos, ya que  pueden distorsionar la media y la mediana, afectando la correcta imputación de los valores nulos.

In [ ]:
 # Observar las medidas estadísticas
df[df_num].describe().T
# Comprobar la presencia de valores atípicos comparando los histogramas con los diagramas de cajas de cada columna
sp.subplot_col_num(df, df_num)
# Detectar valores atípicos en cinco columnas. Calcular su porcentaje en cada columna para decidir que hacer con ellos.
dicc_outliers = {'avg_monthly_gb_download':23,
                 'extra_data_charges':1,
                 'total_charges':4000 ,
                 'average_monthly_expenses':80} 
for col, out in dicc_outliers.items():
  outliers = df[col][df[col] > out].count()
  print(f'Para la columna {col.upper()} tenemos {outliers}, lo que representa un {round(outliers/df.shape[0] *100,3)}% de los datos')

Aunque algunas variables presentan valores atípicos, su porcentaje es bajo y tienen sentido en el contexto del análisis. Modificarlos podría eliminar información valiosa sobre clientes con patrones de consumo específicos, por lo que decidimos dejarlos tal cual.

## Gestión de nulos

In [ ]:
# Observar columnas con nulos y su porcentaje
col_con_nulos = df.columns[df.isnull().any()]
columnas_nulos_info = pd.DataFrame({
        "columna": col_con_nulos,
        "NumeroNulos": [df[col].isnull().sum() for col in col_con_nulos],
        "PorentajeNulos": [(df[col].isnull().sum()/df.shape[0])*100 for col in col_con_nulos]})
display(columnas_nulos_info)

'total_charges' tenía un porcentaje muy pequeño de valores nulos (0.2%). Por un lado, rellenamos con 0 para los clientes que tienen 'tenure = 0', porque aún no han acumulado cargos. Para los demás casos, usamos la mediana en lugar del promedio, ya que nos ayuda a evitar que se distorsionen los datos.

In [ ]:
# Imputar nulos en 'total_charges'
df.loc[df['customer_tenure_(in_months)']==0, 'total_charges'] = 0 # Imputar 'total_charges' con 0 para clientes con 'tenure = 0'
median_total_charges = df['total_charges'].median() 
df['total_charges'].fillna(median_total_charges, inplace=True) # Imputar los valores nulos restantes con la mediana

'monthly_charge' tenía un porcentaje muy pequeño de valores nulos (0.2%), por lo que se rellenaron con la mediana, ya que no tenía una relación significativa con otras variables. Esto garantiza la coherencia en los datos sin introducir sesgos ni afectar significativamente el análisis.

In [ ]:
# Imputar nulos en 'monthly_charge' con la mediana
median_monthly_charge = df['monthly_charge'].median()
df['monthly_charge'].fillna(median_monthly_charge, inplace=True)

In [ ]:
df.isnull().sum() # Comprobar que no hay nulos en ninguna columna

In [ ]:
df.to_excel("../data/datos_churn_limpios.xlsx", index=False) # Guardar cambios